In [1]:
from KnnClassifier import Knn
import numpy as np
from sklearn.model_selection import train_test_split 

In [2]:
file = "datingTestSet.txt"

In [3]:
## convert text file into matrix
def file2matrix(filename, delimeter="\t", index_y =-1):
    fr = open(filename)
    numberOfLines = len(fr.readlines())
    fr = open(filename)
    numberOfX = len(fr.readline().split(delimeter))-1
    returnMatX = np.zeros((numberOfLines,numberOfX))
    classLabelVector = []
    fr = open(filename)
    index = 0
    for line in fr.readlines():
        line = line.strip()                            # remove space between lines (rows)
        listFromLine = line.split(delimeter)           # split the line into a list of elements delimited '\t'.
        classLabelVector.append(listFromLine[index_y]) # get target item from the list to put into classLabelVector
        listFromLine.pop(index_y)                      # remove the target item at the specified position       
        returnMatX[index,:] = listFromLine        
        index += 1
    return returnMatX,classLabelVector

In [4]:
returnMatX,classLabelVector = file2matrix(file)
returnMatX.shape

(1000, 3)

In [5]:
distribution ={}
for x in classLabelVector:
    distribution[x] = distribution.get(x,1)+1

distribution   

{'largeDoses': 328, 'smallDoses': 332, 'didntLike': 343}

In [6]:
def scale_data(dataset,method = "standard"):
    if method == "standard":
        meanVal = np.mean(dataset,0)
        stdVal = np.std(dataset,0)
        standardDataset = np.zeros((dataset.shape))
        for i in range(dataset.shape[1]):
            standardDataset[:,i] = np.array([(x-meanVal[i])/stdVal[i] for x in dataset[:,i]]) 
        return standardDataset
    elif method == "normal":
        minVal = np.min(dataset,0)
        maxVal = np.max(dataset,0)
        rangeVal = maxVal - minVal
        normDataset = np.zeros((dataset.shape))
        for i in range(dataset.shape[1]):
            normDataset[:,i] = np.array([(x-minVal[i])/rangeVal[i] for x in dataset[:,i]])
        return normDataset
    else:
        raise Exception("There are only two methods: standard and normal") 

In [7]:
scaledMat = scale_data(returnMatX,method="normal")
scaledMat

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [8]:
numericClass = []
for x in classLabelVector:
    if x == 'largeDoses':
        numericClass.append(0)
    elif x== 'smallDoses':
        numericClass.append(1)
    else:
        numericClass.append(2)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(scaledMat, numericClass, test_size=0.33)

In [14]:
model = Knn(5)
model.fit(X_train, y_train, "jaccard")
print(model.score())
pred = model.prediction(X_test)
SUM = 0
accuracy = [pred==true for pred,true in zip(pred, y_test)].count(True)/len(y_test)
accuracy

0.9746268656716418


0.9363636363636364